In [11]:
# 호텔 기본 정보 추가하기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser


def moneyTrimmer(input_string):
    
    '''
    Hotel Price data would look like "$263 - $505".
    This method change the one to 263. (get minimum price)
    '''
    
    temp_list = input_string.split(" - ")
    return (temp_list[0])[1:].replace(",", "")


def hotelInformationGetter(input_data, offset):
    
    '''
    Get hotel data and extract information.
    '''
    
    # 파라미터 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")
    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
    api_url = ("/hotels/list?offset="
                 +str(offset)
                 +"&currency=USD&child_rm_ages=7%252C10&limit=30&checkin=2020-01-08&order=asc&lang=en_US&sort=recommended&nights=1&location_id="
                 + str(location_id)
                 +"&adults=1&rooms=1")
    
    # &hotel_class=5 -> 5등급만 뽑기
    #print(api_url)
    
    conn.request("GET", api_url, headers=headers)
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    hotels_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            hotel_data = []
            hotel_data.append(one_data.get("name"))
            sub_category = one_data.get("subcategory_type")
            sub_categorys = ["hotel", "small_hotel", "hostel", "guest_house", "villa"]
            if not sub_category in sub_categorys:
                continue
            else:
                hotel_data.append(sub_category)                
            hotel_data.append(int(float(one_data.get("hotel_class"))))
            hotel_data.append(int(moneyTrimmer(one_data.get("price"))))
            hotel_data.append(cityId) # cityId
            hotels_data.append(hotel_data)
        except AttributeError as e:
            continue

    return hotels_data

def sqlController(hotel_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password,
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Hotel"
    sql_insert_hotel_information = "INSERT INTO City (continent, country, name) VALUES('Europe', 'France', 'Paris')"
    sql_insert_hotels_information = "INSERT INTO Hotel (name, category, subCategory, cost, CityId) VALUES(%s, %s, %s, %s, %s)"
    sql_delete = "DELETE FROM Hotel"
    sql_update_price = "UPDATE Hotel SET cost = %s WHERE name = %s"
    
    # 호텔 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    hotel_data = curs.fetchall() 
    #print("hotel_data", hotel_data) # 지금까지 DB에 있는 호텔 데이터
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 호텔 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_hotel_data = []
    overlapped_hotel_cost_data = []
    
    for info in hotel_info_list:
        if not dataOverlapChecker(hotel_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_hotel_data.append(info)
        else: # 겹치는 호텔의 경우 -> cost만 업데이트 한다
            overlapped_hotel_cost_data.append((info[3], info[0]))
    print("overlapped_hotel_cost_data", overlapped_hotel_cost_data)
    print("new_hotel_data", new_hotel_data)
    curs.executemany(sql_insert_hotels_information, new_hotel_data)
    curs.executemany(sql_update_price, overlapped_hotel_cost_data)
    
    #curs.execute(sql_delete) # 테이블 삭제
    #data = curs.fetchall() # 가져오기 (select에서 사용)
    #print(data)
    
    conn.commit() # RDS에 반영하기
    
def dataOverlapChecker(base_data, checking_info):
    '''
    겹치는 데이터가 있는지 여부를 확인하는 메서드
    '''
    
    if (checking_info[0],) in base_data:
        return True
    return False
            

'''
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"usa_newyork" : 60763, "cityId" : 2},
{"france_paris" : 187147, "cityId" : 1},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''

location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
 
    
#for offset in range(0, 270, 30): #그냥돌리면 안되게 방지로 주석해둠
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        hotel_info_list = hotelInformationGetter(location_id, offset)  
        sqlController(hotel_info_list)
    
    
# hotel_info_list = hotelInformationGetter(187147)
# sqlController(hotel_info_list)

[298184, 3]
offset:  0
info ['Mitsui Garden Hotel Gotanda', 'hotel', 3, 105, 3]
info ['Park Hotel Tokyo', 'hotel', 4, 161, 3]
info ['Daiwa Roynet Hotel Ginza', 'hotel', 3, 134, 3]
info ['Hotel Sardonyx Ueno', 'hotel', 3, 79, 3]
info ['Mitsui Garden Hotel Ginza Premier', 'hotel', 4, 159, 3]
info ['Hotel East 21 Tokyo', 'hotel', 4, 79, 3]
info ['Mitsui Garden Hotel Ginza Gochome', 'hotel', 3, 135, 3]
info ['Shinagawa Prince Hotel', 'hotel', 3, 123, 3]
info ['Keikyu EX Hotel Shinagawa', 'hotel', 3, 101, 3]
info ['Millennium Mitsui Garden Hotel Tokyo', 'hotel', 4, 169, 3]
info ['The Royal Park Hotel Tokyo Shiodome', 'hotel', 4, 170, 3]
info ['Hotel Hokke Club Asakusa', 'hotel', 3, 58, 3]
info ['Tobu Hotel Levant Tokyo', 'hotel', 3, 112, 3]
info ['Hotel Intergate Tokyo Kyobashi', 'hotel', 3, 91, 3]
info ['Tokyu Stay Ginza', 'hotel', 3, 120, 3]
info ['Hotel Niwa Tokyo', 'hotel', 4, 106, 3]
overlapped_hotel_cost_data [(83, 'Hotel Sunroute Higashi Shinjuku'), (141, 'Hotel Gracery Shinjuku'), (

[293913, 4]
offset:  30
overlapped_hotel_cost_data [(23, 'Meander Taipei Hostel'), (112, 'Brother Hotel Taipei'), (164, 'Les Suites Taipei Ching-cheng'), (133, 'Swiio Hotel Daan'), (132, 'Artree Hotel'), (212, 'Grand Mayfull Hotel Taipei'), (112, 'amba Taipei Songshan'), (341, 'W Taipei'), (217, 'The Okura Prestige Taipei'), (146, 'Hotel Eclat Taipei'), (45, 'Eastin Taipei Hotel'), (65, 'CityInn Hotel - Taipei Station Branch I'), (286, 'The Gaia Hotel, Taipei'), (64, 'CityInn Hotel - Taipei Station Branch III'), (107, 'Pacific Business Hotel'), (77, 'Via Hotel'), (85, 'citizenM Taipei North Gate'), (80, 'Just Palace Hotel'), (49, "Ark Hotel - Chang'an Fuxing"), (47, 'Ximen - Airline Hotel'), (162, 'Hotel DOUBLE ONE'), (248, "Shangri-La's Far Eastern Plaza Hotel Taipei"), (63, 'UiNN Business Hotel'), (153, 'Madison Taipei Hotel'), (57, 'Click Hotel - Taipei Main Station Branch'), (47, 'Moshamanla Hotel-Main Station'), (83, 'United Hotel')]
new_hotel_data []
[298085, 5]
offset:  30
overl

[298085, 5]
offset:  60
overlapped_hotel_cost_data [(138, 'Mercure Danang French Village Bana Hills Hotel'), (53, 'New Orient Hotel'), (75, 'Grand Tourane Hotel'), (96, 'Hilton Da Nang'), (47, 'Muong Thanh Grand Da Nang Hotel'), (43, 'Samdi Hotel Da Nang'), (11, 'Danang Backpackers Hostel'), (38, 'Fivitel Da Nang Hotel'), (40, 'SenRiver Hotel'), (37, 'Temple Da Nang Resort Experience'), (37, 'Fivitel Boutique Da Nang'), (57, 'The Blossom Premier Beach Front - Onsen & Foot Massage Inclusive'), (33, 'Dong Duong Hotel & Suites'), (62, 'Muong Thanh Luxury Danang Hotel'), (63, 'Holiday Beach Danang Hotel & Resort'), (30, 'Avora Boutique Hotel'), (30, 'Sunny Ocean Hotel & Spa'), (15, 'Royal Charm Hotel'), (91, 'Son Tra Resort & Spa'), (18, 'Nhat Linh Danang Hotel'), (71, 'DLG Hotel'), (24, "Queen's Finger Hotel"), (23, 'Cordial Hotel'), (23, 'Sea Corner Boutique Hotel'), (23, 'NAM Hotel & Spa')]
new_hotel_data []
[60763, 2]
offset:  60
overlapped_hotel_cost_data [(190, 'Ace Hotel New York'),

In [32]:
########## 5성급만 뽑도록!! -> 근데 바꾼거로는 안해도 ㄱㅊ을듯?

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser


def moneyTrimmer(input_string):
    
    '''
    Hotel Price data would look like "$263 - $505".
    This method change the one to 263. (get minimum price)
    '''
    
    temp_list = input_string.split(" - ")
    return (temp_list[0])[1:].replace(",", "")


def hotelInformationGetter(input_data, offset):
    
    '''
    Get hotel data and extract information.
    '''
    
    # 파라미터 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")
    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
    api_url = ("/hotels/list?offset="
                 +str(offset)
                 +"&currency=USD&child_rm_ages=7%252C10&limit=30&checkin=2020-01-08&hotel_class=0&order=asc&lang=en_US&sort=recommended&nights=1&location_id="
                 + str(location_id)
                 +"&adults=1&rooms=1")
    
    conn.request("GET", api_url, headers=headers)
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    hotels_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            hotel_data = []
            hotel_data.append(one_data.get("name"))
            sub_category = one_data.get("subcategory_type")
            sub_categorys = ["hotel", "small_hotel", "hostel", "guest_house", "villa"]
            if not sub_category in sub_categorys:
                continue
            else:
                hotel_data.append(sub_category)                
            hotel_data.append(int(float(one_data.get("hotel_class"))))
            hotel_data.append(int(moneyTrimmer(one_data.get("price"))))
            hotel_data.append(cityId) # cityId
            hotels_data.append(hotel_data)
        except AttributeError as e:
            continue

    return hotels_data

def sqlController(hotel_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password,
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Hotel"
    sql_insert_hotel_information = "INSERT INTO City (continent, country, name) VALUES('Europe', 'France', 'Paris')"
    sql_insert_hotels_information = "INSERT INTO Hotel (name, category, subCategory, cost, CityId) VALUES(%s, %s, %s, %s, %s)"
    sql_delete = "DELETE FROM Hotel"
    sql_update_price = "UPDATE Hotel SET cost = %s WHERE name = %s"
    
    # 호텔 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    hotel_data = curs.fetchall() 
    #print("hotel_data", hotel_data) # 지금까지 DB에 있는 호텔 데이터
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 호텔 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_hotel_data = []
    overlapped_hotel_cost_data = []
    
    for info in hotel_info_list:
        if not dataOverlapChecker(hotel_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_hotel_data.append(info)
        else: # 겹치는 호텔의 경우 -> cost만 업데이트 한다
            overlapped_hotel_cost_data.append((info[3], info[0]))
    print("overlapped_hotel_cost_data", overlapped_hotel_cost_data)
    print("new_hotel_data", new_hotel_data)
    curs.executemany(sql_insert_hotels_information, new_hotel_data)
    curs.executemany(sql_update_price, overlapped_hotel_cost_data)
    
    #curs.execute(sql_delete) # 테이블 삭제
    #data = curs.fetchall() # 가져오기 (select에서 사용)
    #print(data)
    
    conn.commit() # RDS에 반영하기
    
def dataOverlapChecker(base_data, checking_info):
    '''
    겹치는 데이터가 있는지 여부를 확인하는 메서드
    '''
    
    if (checking_info[0],) in base_data:
        return True
    return False
            

'''
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"usa_newyork" : 60763, "cityId" : 2},
{"france_paris" : 187147, "cityId" : 1},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''

location_id_list = [[298184, 3], [293913, 4], [298085, 5], [294261, 6]]
 
    
for offset in range(360, 420, 30): #그냥돌리면 안되게 방지로 주석해둠
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        hotel_info_list = hotelInformationGetter(location_id, offset)  
        sqlController(hotel_info_list)


[298184, 3]
offset:  360
info ['Hotel Sunroute Stellar Ueno', 'hotel', 3, 90, 3]
info ['Asakusa Central Hotel', 'hotel', 3, 60, 3]
info ['Keikyu EX Hotel Takanawa', 'hotel', 3, 101, 3]
info ['Smile Hotel Tokyo Nihombashi', 'hotel', 3, 48, 3]
info ['Hotel Villa Fontaine Grand Tamachi', 'hotel', 3, 67, 3]
info ['APA Hotel Kanda Jimbocho Eki Higashi', 'hotel', 3, 53, 3]
info ['JR-East Hotel Mets Akabane', 'hotel', 3, 64, 3]
info ['Hotel Guest1 Uenoekimae', 'hotel', 3, 83, 3]
info ['Tsuki', 'hotel', 3, 135, 3]
info ['ICI Hotel Asakusabashi by Relief', 'hotel', 3, 64, 3]
info ['Keikyu EX Inn Shinbanba-Station North', 'hotel', 3, 53, 3]
info ['Ueno Hotel', 'hotel', 3, 52, 3]
info ['Suidobashi Grand Hotel', 'hotel', 3, 71, 3]
info ['Daiwa Roynet Hotel Shimbashi', 'hotel', 3, 91, 3]
overlapped_hotel_cost_data [(34, 'Tokyo Central Youth Hostel'), (66, 'OYO 471 Hotel New Washington'), (56, 'Super Hotel Shinagawa Aomonoyokocho'), (27, 'Hotel Tetora Akabane'), (51, 'Hotel Tetora Ikebukuro')]
new_h

info ['Bamboo House Resort Moalboal', 'hostel', 0, 13, 6]
info ['Casa Del Mar Beach Hotel', 'hotel', 3, 41, 6]
overlapped_hotel_cost_data [(14, 'GV Hotel Lapu-Lapu'), (18, 'Resort Cebu'), (47, "Hisoler's Beach Resort"), (33, 'Punta House'), (40, 'Days Hotel by Wyndham Cebu-Toledo'), (11, 'Capitol Tourist Inn')]
new_hotel_data [['Bamboo House Resort Moalboal', 'hostel', 0, 13, 6], ['Casa Del Mar Beach Hotel', 'hotel', 3, 41, 6]]


In [81]:
# 식당 정보 기본으로 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

def moneyTrimmer(input_string):
    '''
    Food Price data would look like "$20 - $40".
    This method change the one to 30.
    This element will be inserted in cost column.
    '''
    temp_list = input_string.replace(",", "").split(" - ")
    if len(temp_list) == 1:
        return int(temp_list[0][1:])
    mean = (int)((int(temp_list[0][1:])+int(temp_list[1][1:]))/2)
    
    return mean

def priceLevelTrimmer(input_price_level):
    '''
    Price level data would look like $$ - $$$.
    This method change the one to 2.
    $ -> 1
    $$ - $$$ -> 2
    $$$$ -> 3
    This element will be inserted in grade column.
    '''
    if input_price_level == None:
        return None
    
    if input_price_level.count("$") == 1:
        return 1
    elif input_price_level.count("$") == 5:
        return 2
    else:
        return 3
    
def restaurantInformationGetter(input_data, offset):
    '''
    Get restaurant data and extract information.
    '''
    
    # 파라미터 변수 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
        
    conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
                 + str(offset)
                 + "&location_id="
                 + str(location_id),
                 headers=headers)

    # prices_restaurants=10953%252C10954&  -> 1과 3단계만 넣기.
    
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    restaurants_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            restaurant_data = []
            if one_data.get("name") == None:
                continue
            restaurant_data.append(one_data.get("name"))
            restaurant_data.append(priceLevelTrimmer(one_data.get("price_level"))) # grade
            if -1 == one_data.get("price", -1): # cost가 없으면 그 데이터는 넣지 않는다.
                continue
            elif 1 == int(moneyTrimmer(one_data.get("price"))): # cost가 1이면 안넣기
                continue
            else:
                restaurant_data.append(int(moneyTrimmer(one_data.get("price")))) # 넣기
            restaurant_data.append(cityId) # cityId
            restaurants_data.append(restaurant_data)
        except AttributeError as e:
            continue
    return restaurants_data

def sqlController(food_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user ,password=db_password, 
                           db=db_name, charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Food"
    sql_insert_foods_information = "INSERT INTO Food (name, grade, cost, CityId) VALUES(%s, %s, %s, %s)"
    sql_delete = "DELETE FROM Food"
    sql_update_price = "UPDATE Food SET cost = %s WHERE name = %s"
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    food_data = curs.fetchall() 
    #print("food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_food_data = []
    overlapped_food_cost_data = []
    for info in food_info_list:
        if not dataOverlapChecker(food_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_food_data.append(info)
        else: # 겹치는 food의 경우 -> cost만 업데이트 한다
            overlapped_food_cost_data.append((info[3], info[0]))
    print("overlapped_food_cost_data", overlapped_food_cost_data)
    print("new_food_data", new_food_data)
    curs.executemany(sql_insert_foods_information, new_food_data)
    curs.executemany(sql_update_price, overlapped_food_cost_data)

    conn.commit() # RDS에 반영하기


'''
{"france_paris" : 187147, "cityId" : 1},
{"usa_newyork" : 60763, "cityId" : 2},
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''
location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]


for offset in range(90, 360, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        restaurant_info_list = restaurantInformationGetter(location_id, offset)  
        sqlController(restaurant_info_list)
    

[298184, 3]
offset:  90
info ['CICADA', 2, 33, 3]
info ['Butagumi', 2, 37, 3]
info ['Tsuru Ton Tan Udon Noodle Brasserie, Ginza', 2, 27, 3]
info ['Gyozanofukuho', 2, 8, 3]
info ['Rigoletto Wine and Bar', 2, 23, 3]
info ['Tachiguimidori, Echika Ikebukuro', 2, 8, 3]
info ['The Great Burger', 2, 18, 3]
info ['Ananda', 2, 50, 3]
info ['IVY PLACE', 2, 31, 3]
info ['AIN SOPH. Journey Shinjuku', 2, 25, 3]
info ['Nabezo Asakusa Kaminarimon', 2, 28, 3]
info ['Shinjuku Kakekomi Gyoza kabukichou', 2, 18, 3]
info ['IPPUDO Ebisu', 2, 8, 3]
info ['T.Y.HARBOR', 2, 27, 3]
info ['Asakusa Umaimon Aduma', 2, 18, 3]
info ['Tsurutontan Roppongi', 2, 8, 3]
overlapped_food_cost_data []
new_food_data [['CICADA', 2, 33, 3], ['Butagumi', 2, 37, 3], ['Tsuru Ton Tan Udon Noodle Brasserie, Ginza', 2, 27, 3], ['Gyozanofukuho', 2, 8, 3], ['Rigoletto Wine and Bar', 2, 23, 3], ['Tachiguimidori, Echika Ikebukuro', 2, 8, 3], ['The Great Burger', 2, 18, 3], ['Ananda', 2, 50, 3], ['IVY PLACE', 2, 31, 3], ['AIN SOPH. Journ

info ['The Shrimp Daddy', 2, 5, 4]
info ['Antico Forno Pizza', 2, 500, 4]
info ["O'Steak", 2, 32, 4]
info ['Thai Town Cuisine- Taipei Main Station Mitsukoshi Store', 2, 18, 4]
info ['SANHOYAN - ZhongXiao', 2, 15, 4]
info ['Garden Kitchen', 2, 72, 4]
overlapped_food_cost_data []
new_food_data [['The Shrimp Daddy', 2, 5, 4], ['Antico Forno Pizza', 2, 500, 4], ["O'Steak", 2, 32, 4], ['Thai Town Cuisine- Taipei Main Station Mitsukoshi Store', 2, 18, 4], ['SANHOYAN - ZhongXiao', 2, 15, 4], ['Garden Kitchen', 2, 72, 4]]
[298085, 5]
offset:  120
info ['Lotus Wine & Dine Restaurant', 2, 10, 5]
info ['Am Thuc Xeo Vo Van Kiet', 1, 94500, 5]
info ['Quan Hue Ngon', 1, 2, 5]
info ['Issun Boushi(一寸法師2)Japan Restaurant', 2, 22, 5]
info ['The Amazing Taco', 2, 5, 5]
info ['Tandoori~Nights', 1, 6, 5]
info ['Mai Boutique Restaurant & Bar', 2, 10, 5]
info ['Madame Flip Flop Cafe', 2, 5, 5]
info ['Hai San Be Man', 2, 6, 5]
info ["AN's - Vegeterian Cusines", 1, 3, 5]
info ["Dog Star's", 2, 7, 5]
info ['Ann

info ['Vezzo', 2, 10, 2]
info ['Cuba', 2, 30, 2]
info ["Forlini's Restaurant", 2, 20, 2]
info ['Trattoria Trecolori', 2, 62, 2]
info ['Etcetera Etcetera', 2, 57, 2]
info ["Don's Bogam BBQ & Wine Bar", 2, 30, 2]
info ['Pearl Oyster Bar', 2, 40, 2]
info ["Arturo's", 2, 30, 2]
info ['Blue Smoke Battery Park City', 2, 30, 2]
info ['Pepe Giallo Italian Restaurant', 2, 22, 2]
info ['Blue Dog Cookhouse & Bar', 2, 18, 2]
info ['12 Chairs', 2, 15, 2]
info ['VIV Thai', 2, 21, 2]
info ['OCabanon', 2, 20, 2]
info ['Toloache', 2, 60, 2]
info ['Market Table', 2, 30, 2]
info ['Brasserie 8 1/2', 2, 55, 2]
info ['Da Claudio', 2, 20, 2]
info ['Cookshop', 2, 60, 2]
info ['Dirt Candy', 2, 40, 2]
overlapped_food_cost_data []
new_food_data [['Vezzo', 2, 10, 2], ['Cuba', 2, 30, 2], ["Forlini's Restaurant", 2, 20, 2], ['Trattoria Trecolori', 2, 62, 2], ['Etcetera Etcetera', 2, 57, 2], ["Don's Bogam BBQ & Wine Bar", 2, 30, 2], ['Pearl Oyster Bar', 2, 40, 2], ["Arturo's", 2, 30, 2], ['Blue Smoke Battery Park Ci

info ['Le Poulbot', 2, 21, 1]
info ['Hao Long', 2, 26, 1]
info ['Le Recepteur', 2, 28, 1]
info ['Le Sens Unique', 2, 29, 1]
info ['La Traversee', 2, 23, 1]
info ['Il Farniente', 2, 25, 1]
info ['Couscous Maison Arc en Ciel', 1, 14, 1]
info ["Les Tables d'Augustin", 2, 29, 1]
info ['Schum', 2, 30, 1]
info ['Bien Ficele', 2, 33, 1]
info ["Bistrot de L'Oulette", 2, 32, 1]
info ['AU VIEUX COMPTOIR', 2, 62, 1]
info ['Meating Corner', 1, 17, 1]
info ['IZU', 2, 24, 1]
info ['Grains Nobles et Plus', 2, 33, 1]
info ['La Jacobine', 2, 22, 1]
info ['Little Cantine - Burgers & Bakery', 2, 17, 1]
info ['Le Maharaja', 2, 33, 1]
info ['Le Pot de Vins', 2, 18, 1]
info ["L'Alsacien", 2, 22, 1]
overlapped_food_cost_data []
new_food_data [['Le Poulbot', 2, 21, 1], ['Hao Long', 2, 26, 1], ['Le Recepteur', 2, 28, 1], ['Le Sens Unique', 2, 29, 1], ['La Traversee', 2, 23, 1], ['Il Farniente', 2, 25, 1], ['Couscous Maison Arc en Ciel', 1, 14, 1], ["Les Tables d'Augustin", 2, 29, 1], ['Schum', 2, 30, 1], ['Bie

info ['Menya Kaijin Shinjuku', 2, 8, 3]
info ['THE APOLLO', 2, 45, 3]
info ['Din Tai Fung (Tamagawa Takashimaya Branch)', 2, 37, 3]
info ['Palermo Pizza & Pasta Akasaka Branch', 2, 18, 3]
info ['DELHI', 2, 8, 3]
info ['Kyourakutei', 2, 8, 3]
info ['Hamburg Will', 2, 18, 3]
info ['Halal Wagyu Yakiniku PANGA', 2, 42, 3]
info ['Negishi Shinjuku L tower ten', 2, 8, 3]
info ['Yonekyu', 2, 37, 3]
info ['Yakiniku Happy Ningyocho', 2, 27, 3]
info ['Union Square Tokyo', 2, 18, 3]
info ['Nikugen, Akasaka', 2, 55, 3]
info ['Yoshinoya', 1, 8, 3]
info ['Tempura Shinjuku Tsunahachi Ginza', 2, 18, 3]
overlapped_food_cost_data []
new_food_data [['Menya Kaijin Shinjuku', 2, 8, 3], ['THE APOLLO', 2, 45, 3], ['Din Tai Fung (Tamagawa Takashimaya Branch)', 2, 37, 3], ['Palermo Pizza & Pasta Akasaka Branch', 2, 18, 3], ['DELHI', 2, 8, 3], ['Kyourakutei', 2, 8, 3], ['Hamburg Will', 2, 18, 3], ['Halal Wagyu Yakiniku PANGA', 2, 42, 3], ['Negishi Shinjuku L tower ten', 2, 8, 3], ['Yonekyu', 2, 37, 3], ['Yakinik

info ['Cua Hang An Uong Mau Dich 1986', 2, 3, 5]
info ['Kokogreen Kafe', 2, 8, 5]
info ['Sen 3 Mien Restaurant', 2, 252, 5]
info ['Shamrock Sports Bar Da Nang', 1, 4, 5]
info ['Indochina Riverside Food Court', 2, 4, 5]
info ['The Hotdog Station DaNang', 1, 2, 5]
info ['City High Restaurant', 1, 5, 5]
info ['The Secret Garden Pub & Cafe', 1, 4, 5]
info ['The Veranda Grill', 2, 17, 5]
info ['Dak Gal Bi Restaurant', 1, 7, 5]
info ['Huck Restaurant', 1, 8, 5]
info ['An Nam Restaurant', 2, 19, 5]
info ['SSAM DAK - Korean Restaurant', 2, 13, 5]
info ['Cha Chang Da Nang', 1, 3, 5]
info ['Ngu Sen Garden', 1, 4, 5]
info ['MANA Garden Restaurant', 2, 11, 5]
info ['Apsara', 2, 7, 5]
info ['Cuisine Pallet BBQ', 1, 4, 5]
overlapped_food_cost_data []
new_food_data [['Cua Hang An Uong Mau Dich 1986', 2, 3, 5], ['Kokogreen Kafe', 2, 8, 5], ['Sen 3 Mien Restaurant', 2, 252, 5], ['Shamrock Sports Bar Da Nang', 1, 4, 5], ['Indochina Riverside Food Court', 2, 4, 5], ['The Hotdog Station DaNang', 1, 2, 5],

info ['Salento Montmartre', 2, 27, 1]
info ['Cedars', 1, 13, 1]
info ['Creperie Gigi', 2, 25, 1]
info ['Murier', 2, 30, 1]
info ['Lucette fait des Crepes', 1, 20, 1]
info ['Meriggio', 2, 35, 1]
info ['Le 14 Juillet', 2, 22, 1]
info ['Casta Diva', 2, 15, 1]
info ['Paris-Breizh Place d’Italie', 2, 21, 1]
info ['The Frog Revolution', 2, 17, 1]
info ['Le Verre Moutarde', 2, 52, 1]
info ['Le Timbre', 2, 42, 1]
info ['Sourire Tapas Francaises', 2, 36, 1]
info ['Ristorante Adaggio', 2, 25, 1]
info ["L'Alchimiste", 2, 25, 1]
overlapped_food_cost_data []
new_food_data [['Salento Montmartre', 2, 27, 1], ['Cedars', 1, 13, 1], ['Creperie Gigi', 2, 25, 1], ['Murier', 2, 30, 1], ['Lucette fait des Crepes', 1, 20, 1], ['Meriggio', 2, 35, 1], ['Le 14 Juillet', 2, 22, 1], ['Casta Diva', 2, 15, 1], ['Paris-Breizh Place d’Italie', 2, 21, 1], ['The Frog Revolution', 2, 17, 1], ['Le Verre Moutarde', 2, 52, 1], ['Le Timbre', 2, 42, 1], ['Sourire Tapas Francaises', 2, 36, 1], ['Ristorante Adaggio', 2, 25, 1]

In [83]:
# 식당 3 등급만 넣기.

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

def moneyTrimmer(input_string):
    '''
    Food Price data would look like "$20 - $40".
    This method change the one to 30.
    This element will be inserted in cost column.
    '''
    temp_list = input_string.replace(",", "").split(" - ")
    if len(temp_list) == 1:
        return int(temp_list[0][1:])
    mean = (int)((int(temp_list[0][1:])+int(temp_list[1][1:]))/2)
    
    return mean

def priceLevelTrimmer(input_price_level):
    '''
    Price level data would look like $$ - $$$.
    This method change the one to 2.
    $ -> 1
    $$ - $$$ -> 2
    $$$$ -> 3
    This element will be inserted in grade column.
    '''
    if input_price_level == None:
        return None
    
    if input_price_level.count("$") == 1:
        return 1
    elif input_price_level.count("$") == 5:
        return 2
    else:
        return 3
    
def restaurantInformationGetter(input_data, offset):
    '''
    Get restaurant data and extract information.
    '''
    
    # 파라미터 변수 정리
    location_id = input_data[0]
    cityId = input_data[1]
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    rapid_api_key = config['API']['tripadvisorapi']
    
    # rapidapi 연결하기
    conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

    headers = {
        'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
        'x-rapidapi-key': rapid_api_key
        }
    # 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
        
    conn.request("GET", "/restaurants/list?lunit=km&limit=30&prices_restaurants=10954&currency=USD&offset="
                 + str(offset)
                 + "&lang=en_US&location_id="
                 + str(location_id),
                 headers=headers)
    
    # prices_restaurants=10953%252C10954&  -> 1과 3단계만 넣기. 지금은 3단계만 넣는 것으로 되어있음. 그리고 저 주석이 맞는지 모르겠음.
    
    res = conn.getresponse()
    
    # response 읽기
    data = res.read()

    # response를 dictionary로 만들기
    dict_data = json.loads(data)
    #print(dict_data) # 확인 출력문
    
    # 딕셔너리 다듬기
    dict_data_data = dict_data.get('data')

    #price_list = [] # 호텔의 가격을 담아둘 리스트 (우선은 안쓰고 추후에 가운데 값 구할 때 사용?)
    restaurants_data = [] # 테이블에 들어갈 호텔들의 정보를 담아두는 리스트 (이름, 클래스, 가격, 카테고리?)
    for one_data in dict_data_data:
        try:
            restaurant_data = []
            if one_data.get("name") == None:
                continue
            restaurant_data.append(one_data.get("name"))
            restaurant_data.append(priceLevelTrimmer(one_data.get("price_level"))) # grade
            
            if -1 == one_data.get("price", -1): # cost가 없으면 그 데이터는 넣지 않는다.
                continue
            elif 1 == int(moneyTrimmer(one_data.get("price"))): # cost가 1이면 안넣기
                continue
            else:
                restaurant_data.append(int(moneyTrimmer(one_data.get("price")))) # 넣기
                
            restaurant_data.append(cityId) # cityId
            restaurants_data.append(restaurant_data)
        except AttributeError as e:
            continue
    return restaurants_data

def sqlController(food_info_list):
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user ,password=db_password, 
                           db=db_name, charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql = "select name from Food"
    sql_insert_foods_information = "INSERT INTO Food (name, grade, cost, CityId) VALUES(%s, %s, %s, %s)"
    sql_delete = "DELETE FROM Food"
    sql_update_price = "UPDATE Food SET cost = %s WHERE name = %s"
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql) 
    food_data = curs.fetchall() 
    #print("food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    new_food_data = []
    overlapped_food_cost_data = []
    for info in food_info_list:
        if not dataOverlapChecker(food_data, info): # 겹치는 호텔정보가 아닐 경우
            print("info", info)
            new_food_data.append(info)
        else: # 겹치는 food의 경우 -> cost만 업데이트 한다
            overlapped_food_cost_data.append((info[3], info[0]))
    print("overlapped_food_cost_data", overlapped_food_cost_data)
    print("new_food_data", new_food_data)
    curs.executemany(sql_insert_foods_information, new_food_data)
    curs.executemany(sql_update_price, overlapped_food_cost_data)

    conn.commit() # RDS에 반영하기


'''
{"france_paris" : 187147, "cityId" : 1},
{"usa_newyork" : 60763, "cityId" : 2},
{"japan_tokyo" : 298184, "cityId" : 3},
{"taiwan_taipei" : 293913, "cityId" : 4},
{"vietnam_danang" : 298085, "cityId" : 5},
{"philippines_cebu" : 294261, "cityId" : 6}
[[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]
'''
location_id_list = [[298184, 3], [293913, 4], [298085, 5], [60763, 2], [187147, 1], [294261, 6]]

for offset in range(0, 90, 30):
    for location_id in location_id_list:
        print(location_id)
        print("offset: ", offset)
        restaurant_info_list = restaurantInformationGetter(location_id, offset)  
        sqlController(restaurant_info_list)
    

[298184, 3]
offset:  0
info ['Rokukakutei Ginza', 3, 110, 3]
info ['Ise Sueyoshi', 3, 96, 3]
info ['Han no Daidokoro Honten', 3, 50, 3]
info ['Tapas Molecular Bar', 3, 215, 3]
info ['Yakiniku a Five Toku Ginza8chome', 3, 78, 3]
info ['Towers', 3, 3, 3]
info ['Irish Pub Celts Shibuya Center Gai', 3, 23, 3]
info ['Steak House Pound Akihabara', 3, 59, 3]
info ['Hakushu', 3, 105, 3]
info ['NARISAWA', 3, 303, 3]
info ['Atelier Morimoto XEX', 3, 83, 3]
info ['Imahan Honten', 3, 37, 3]
info ['Le Chateau de Joel Robuchon', 3, 198, 3]
info ['Omotesando Ukaitei', 3, 197, 3]
info ['Sushi Bar Yasuda', 3, 82, 3]
info ['Ginza Seamon', 3, 92, 3]
info ['Kobe Beef Kaiseki 511', 3, 125, 3]
overlapped_food_cost_data []
new_food_data [['Rokukakutei Ginza', 3, 110, 3], ['Ise Sueyoshi', 3, 96, 3], ['Han no Daidokoro Honten', 3, 50, 3], ['Tapas Molecular Bar', 3, 215, 3], ['Yakiniku a Five Toku Ginza8chome', 3, 78, 3], ['Towers', 3, 3, 3], ['Irish Pub Celts Shibuya Center Gai', 3, 23, 3], ['Steak House Pound

info ['Teppanyaki Akasaka', 3, 114, 3]
info ['NINJA AKASAKA', 3, 112, 3]
info ['The Peak Lounge', 3, 18, 3]
info ['Oriental Lounge', 3, 50, 3]
info ["L'Atelier de Joel Robuchon", 3, 298, 3]
info ['Sense', 3, 94, 3]
info ["Lawry's the Prime Rib, Ebisu", 3, 78, 3]
info ['Pierre Gagnaire', 3, 109, 3]
info ['Hinokizaka', 3, 5, 3]
info ['Cuisine(S) Michel Troisgros', 3, 183, 3]
info ['Signature', 3, 2, 3]
info ['The Imperial Viking Sal', 3, 69, 3]
info ["L'Effervescence", 3, 94, 3]
info ['Azure 45', 3, 3, 3]
info ['Beige Alain Ducasse Tokyo', 3, 300, 3]
overlapped_food_cost_data [(3, 'Rokkasen'), (3, 'Les Saisons')]
new_food_data [['Teppanyaki Akasaka', 3, 114, 3], ['NINJA AKASAKA', 3, 112, 3], ['The Peak Lounge', 3, 18, 3], ['Oriental Lounge', 3, 50, 3], ["L'Atelier de Joel Robuchon", 3, 298, 3], ['Sense', 3, 94, 3], ["Lawry's the Prime Rib, Ebisu", 3, 78, 3], ['Pierre Gagnaire', 3, 109, 3], ['Hinokizaka', 3, 5, 3], ['Cuisine(S) Michel Troisgros', 3, 183, 3], ['Signature', 3, 2, 3], ['The 

info ['Momoyama - Sheraton Grand Taipei Hotel', 3, 62, 4]
info ['Marshal Zen Garden', 3, 2150, 4]
info ['J. W. Teres, The Bulgarian Restaurant', 3, 42, 4]
info ['Da Antonio Ristorante Italiano', 3, 25, 4]
info ["Mark's Teppanyaki", 3, 216, 4]
info ['Shintori Restaurant', 3, 93, 4]
info ['Ibuki - Shangri-La', 3, 60, 4]
info ['Imperial Steak House', 3, 66, 4]
info ['Mitsui', 3, 62, 4]
info ['Eclat Lounge', 3, 42, 4]
info ['A Cut Steakhouse', 3, 62, 4]
info ['Rilakkuma Cha-Ya', 3, 9, 4]
info ['TK Seafood & Steak', 3, 33, 4]
overlapped_food_cost_data []
new_food_data [['Momoyama - Sheraton Grand Taipei Hotel', 3, 62, 4], ['Marshal Zen Garden', 3, 2150, 4], ['J. W. Teres, The Bulgarian Restaurant', 3, 42, 4], ['Da Antonio Ristorante Italiano', 3, 25, 4], ["Mark's Teppanyaki", 3, 216, 4], ['Shintori Restaurant', 3, 93, 4], ['Ibuki - Shangri-La', 3, 60, 4], ['Imperial Steak House', 3, 66, 4], ['Mitsui', 3, 62, 4], ['Eclat Lounge', 3, 42, 4], ['A Cut Steakhouse', 3, 62, 4], ['Rilakkuma Cha-Ya'

In [86]:
# 호텔 중앙값 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

# outlier 제거하는 메소드
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]


def hotelMedianController():
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password, 
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql_select_hotel = "select * from Hotel"
    sql_select_median = "select * from Median"
    sql_insert_median_data = "INSERT INTO Median (cityId, category, cost, urlHotel) VALUES(%s, %s, %s, %s)"
    sql_update_median_data = "UPDATE Median SET cost = %s, urlHotel = %s WHERE cityId = %s AND category = %s"
    
    # 테이블에 들어갈 url
    url_paris = "https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html"
    url_newyork = "https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html"
    url_tokyo = "https://www.tripadvisor.co.kr/Hotels-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html"
    url_taipei = "https://www.tripadvisor.co.kr/Hotels-g293913-Taipei.html"
    url_danang = "https://www.tripadvisor.co.kr/Hotels-g298085-Da_Nang.html"
    url_cebu = "https://www.tripadvisor.co.kr/Hotels-g294261-Cebu_Island_Visayas-Hotels.html"
    urls = [url_paris, url_newyork, url_tokyo, url_taipei, url_danang, url_cebu]
    
    # 테이블에 들어갈 음식점 호칭
    hotel_naming = ["저가호텔", "일반호텔", "고급호텔", "최고급호텔"] # 1성과 2성 모두 저가호텔로하기
    
    # Hotel 테이블의 내용을 가져오기 + 출력
    curs.execute(sql_select_hotel) 
    hotel_data = curs.fetchall() 
    #print("@@@hotel_data", hotel_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    cities = [[], [], [], [], [], []]

    for hotel_info in hotel_data:
        if hotel_info[5] == 1:
            cities[0].append(hotel_info)
        elif hotel_info[5] == 2:
            cities[1].append(hotel_info)
        elif hotel_info[5] == 3:
            cities[2].append(hotel_info)
        elif hotel_info[5] == 4:
            cities[3].append(hotel_info)
        elif hotel_info[5] == 5:
            cities[4].append(hotel_info)
        else:
            cities[5].append(hotel_info)
        
    for city in cities:
        temp_list = [[], [], [], [], []]
        for hotel in city:
            if hotel[3] == "0":
                temp_list[0].append(hotel[4])
            elif hotel[3] =="1":
                temp_list[0].append(hotel[4])
            elif hotel[3] =="2":
                temp_list[1].append(hotel[4])
            elif hotel[3] =="3":
                temp_list[2].append(hotel[4])
            elif hotel[3] =="4":
                temp_list[3].append(hotel[4])
            else:
                temp_list[4].append(hotel[4])
        city[:] = temp_list[:]
    #print("cities", cities) # 각 도시별, 등급별 가격이 모임
        
    median = []    
    np_cities = np.array(cities)
    for city_idx, np_city in enumerate(np_cities):
        inner_median = []
        for grade_idx, np_grade in enumerate(np_city):
            inner_median.append(int(round(np.mean(reject_outliers(np.array(np_grade))))))
        median.append(inner_median)
    print("median", median)
        
    # Refactor: 1성과 2성의 평균으로 저가호텔가격을 만들기 (1성이 초저가 였는데 2성과 평균을 내고 초저가 가격은 없엔다))
    hotel_median_infos = []
    for city in median:
        hotel_median_list = [int((city[0] + city[1])/2), *city[2:]]
        hotel_median_infos.append(hotel_median_list)   
        
    # Median 테이블의 내용을 가져오기
    curs.execute(sql_select_median) 
    median_data = curs.fetchall() 
    print("median_data", len(median_data))

    final_median_list = []
    print("len(median_data)", len(median_data) )
    if len(median_data) == 0: # 테이블 비었을 경우 -> insert
        for city_idx, one_city in enumerate(hotel_median_infos):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(city_idx+1)
                temp_list.append(hotel_naming[idx])
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                final_median_list.append(temp_list)  
        print("final_median_list", final_median_list)
        curs.executemany(sql_insert_median_data, final_median_list)
    else: # 테이블 이미 차있는 경우 -> update
        print("median_data", median_data)
        for city_idx, one_city in enumerate(hotel_median_infos):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                temp_list.append(city_idx+1)
                temp_list.append(hotel_naming[idx])
                final_median_list.append(temp_list) 
        print("final_median_list", final_median_list)
        curs.execute("SET SQL_SAFE_UPDATES = 0")
        curs.executemany(sql_update_median_data, final_median_list)
        
    conn.commit() # RDS에 반영하기

    
hotelMedianController()
    

median [[65, 78, 98, 164, 359], [77, 101, 136, 180, 331], [58, 57, 90, 180, 455], [42, 33, 66, 123, 234], [13, 14, 23, 51, 167], [21, 20, 37, 67, 189]]
median_data 42
len(median_data) 42
median_data ((9, 1, '저가호텔', 71, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (10, 1, '일반호텔', 98, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (11, 1, '고급호텔', 164, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (12, 1, '최고급호텔', 359, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (13, 2, '저가호텔', 89, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (14, 2, '일반호텔', 136, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (15, 2, '고급호텔', 180, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (16, 2, '최고급호텔', 331, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.

In [87]:
# 식당 중앙값 넣기

import requests
import json
import http.client
import numpy as np
import pymysql
import configparser

# outlier 제거하는 메소드
def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]


def foodMedianController():
    
    '''
    SQL문을 컨트롤 하는 메서드
    '''
    
    # config 변수 정리
    config = configparser.ConfigParser()
    config.read('config.ini')

    db_host = config['DATABASE']['db_host']
    db_user = config['DATABASE']['db_user']
    db_password = config['DATABASE']['db_password']
    db_name = config['DATABASE']['db_name']
    
    # 스키마와 연결
    conn = pymysql.connect(host=db_host, 
                           user=db_user, 
                           password=db_password, 
                           db=db_name,
                           charset='utf8')
    # 커서 생성
    curs = conn.cursor()
    
    # SQL문들
    sql_select_food = "select * from Food"
    sql_select_median = "select * from Median"
    sql_insert_median_data = "INSERT INTO Median (cityId, category, cost, urlFood) VALUES(%s, %s, %s, %s)"
    sql_update_median_data = "UPDATE Median SET cost = %s, urlFood = %s WHERE cityId = %s AND category = %s"
    
    # 테이블에 들어갈 url
    url_paris = "https://www.tripadvisor.co.kr/Restaurants-g187147-Paris_Ile_de_France.html"
    url_newyork = "https://www.tripadvisor.co.kr/Restaurants-g60763-New_York_City_New_York.html"
    url_tokyo = "https://www.tripadvisor.co.kr/Restaurants-g298184-Tokyo_Tokyo_Prefecture_Kanto-Hotels.html"
    url_taipei = "https://www.tripadvisor.co.kr/Restaurants-g293913-Taipei.html"
    url_danang = "https://www.tripadvisor.co.kr/Restaurants-g298085-Da_Nang.html"
    url_cebu = "https://www.tripadvisor.co.kr/Restaurants-g294261-Cebu_Island_Visayas.html"
    urls = [url_paris, url_newyork, url_tokyo, url_taipei, url_danang, url_cebu]
    
    # 테이블에 들어갈 음식점 호칭
    restaurant_naming = ["간편식", "일반음식점", "고급음식점"]
    
    # Food 테이블의 내용을 가져오기 + 출력
    curs.execute(sql_select_food) 
    food_data = curs.fetchall() 
    #print("@@@food_data", food_data)
    
    # 트립어드바이져 API를 이용해서 가져온 정보가 Food 테이블에 들어있는 정보와 겹치는지 확인 후 아닌 것은 추가 맞는 것은 업데이트
    cities = [[], [], [], [], [], []]

    for food_info in food_data:
        if food_info[4] == 1:
            cities[0].append(food_info)
        elif food_info[4] == 2:
            cities[1].append(food_info)
        elif food_info[4] == 3:
            cities[2].append(food_info)
        elif food_info[4] == 4:
            cities[3].append(food_info)
        elif food_info[4] == 5:
            if food_info[3] > 10000:
                continue
            cities[4].append(food_info)
        else:
            cities[5].append(food_info)
    
    for city in cities:
        temp_list = [[], [], []]
        for restaurant in city:
            if restaurant[2] == "1":
                temp_list[0].append(restaurant[3])
            elif restaurant[2] =="2":
                temp_list[1].append(restaurant[3])
            else:
                temp_list[2].append(restaurant[3])
        city[:] = temp_list[:]
        
    median = []    
    np_cities = np.array(cities)
    for idx, np_city in enumerate(np_cities):
        inner_median = []
        for np_grade in np_city:
            inner_median.append(int(round(np.mean(reject_outliers(np.array(np_grade))))))
        #inner_median.append(idx + 1)
        median.append(inner_median)
    print(median)
        
    # Median 테이블의 내용을 가져오기
    curs.execute(sql_select_median) 
    median_data = curs.fetchall() 
    print("median_data", len(median_data))

    final_median_list = []
    if len(median_data) == 24: # 테이블 비었을 경우 -> insert
        for city_idx, one_city in enumerate(median):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(city_idx+1)
                temp_list.append(restaurant_naming[idx])
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                final_median_list.append(temp_list)  
        print("final_median_list", final_median_list)
        curs.executemany(sql_insert_median_data, final_median_list)
    else: # 테이블 이미 차있는 경우 -> update
        print("median_data", median_data)
        for city_idx, one_city in enumerate(median):
            for idx, one_data in enumerate(one_city):
                temp_list = []
                temp_list.append(one_data)
                temp_list.append(urls[city_idx])
                temp_list.append(city_idx+1)
                temp_list.append(restaurant_naming[idx])
                final_median_list.append(temp_list) 
        curs.executemany(sql_update_median_data, final_median_list)

    conn.commit() # RDS에 반영하기

    
foodMedianController()
    

[[7, 29, 84], [7, 27, 55], [7, 26, 75], [9, 30, 73], [5, 15, 88], [5, 18, 120]]
median_data 42
median_data ((9, 1, '저가호텔', 71, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (10, 1, '일반호텔', 98, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (11, 1, '고급호텔', 164, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (12, 1, '최고급호텔', 359, 'https://www.tripadvisor.co.kr/Hotels-g187147-Paris_Ile_de_France.html', None), (13, 2, '저가호텔', 89, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (14, 2, '일반호텔', 136, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (15, 2, '고급호텔', 180, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (16, 2, '최고급호텔', 331, 'https://www.tripadvisor.co.kr/Hotels-g60763-New_York_City_New_York.html', None), (17, 3, '저가호텔', 57, 'https://www.tripadvisor.co.kr/Hotels-g298184-Tokyo_Tokyo_

In [70]:
# 연습

# {"japan_tokyo_shinjuku" : 1066457, "cityId" : 2},
# {"japan_tokyo_sibuya" : 1066456, "cityId" : 2},
# {"taiwan_taipei" : 293913, "cityId" : 3},
# {"vietnam_danang" : 298085, "cityId" : 4},
# {"usa_newyork" : 60763, "cityId" : 1},
# {"france_paris" : 187147, "cityId" : 0},
# {"philippines_cebu" : 298460, "cityId" : 5}

location_id_dict = [[1066457, 2], [1066456, 2], [293913, 3], [298085, 4], [60763, 1], [187147, 0], [298460, 5]]

    
for location_id in location_id_dict:
    print(location_id)
    # hotel_info_list = hotelInformationGetter(location_id)
    # sqlController(hotel_info_list)
    

[1066457, 2]
[1066456, 2]
[293913, 3]
[298085, 4]
[60763, 1]
[187147, 0]
[298460, 5]


In [70]:
# 연결 연습

import requests
import json
import http.client
import numpy as np
import pymysql

'''
Get restaurant data and extract information.
'''
# rapidapi 연결하기
conn = http.client.HTTPSConnection("tripadvisor1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "tripadvisor1.p.rapidapi.com",
    'x-rapidapi-key': "164b57de3amsh8a6777213843ef5p15508djsn556f2053e4ab"
    }

# 호텔 리퀘스트 만들기 (함수의 파라미터로 들어오는 location_id를 이용하여 국가 선택 + 옵션설정을 원하면 rapidAPI홈페이지에서 설정하기)
offset_number = 0

conn.request("GET", "/restaurants/list?restaurant_tagcategory_standalone=10591&currency=KRW&lunit=km&restaurant_tagcategory=10591&limit=30&prices_restaurants=10953%252C10955&restaurant_mealtype=10598%252C10599&currency=USD&lang=en_US&offset="
             + str(30)
             + "&location_id="
             + str(298085),
             headers=headers)

res = conn.getresponse()

# response 읽기
data = res.read()

# response를 dictionary로 만들기
dict_data = json.loads(data)
print(dict_data) # 확인 출력문

# 딕셔너리 다듬기
dict_data_data = dict_data.get('data')


{'data': [{'location_id': '7057425', 'name': 'Limoncello', 'latitude': '16.066114', 'longitude': '108.22364', 'num_reviews': '895', 'timezone': 'Asia/Ho_Chi_Minh', 'location_string': 'Da Nang', 'photo': {'images': {'small': {'width': '150', 'url': 'https://media-cdn.tripadvisor.com/media/photo-l/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '150'}, 'thumbnail': {'width': '50', 'url': 'https://media-cdn.tripadvisor.com/media/photo-t/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '50'}, 'original': {'width': '1280', 'url': 'https://media-cdn.tripadvisor.com/media/photo-o/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '853'}, 'large': {'width': '1024', 'url': 'https://media-cdn.tripadvisor.com/media/photo-w/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '682'}, 'medium': {'width': '550', 'url': 'https://media-cdn.tripadvisor.com/media/photo-s/12/22/eb/a2/thin-crust-pizza-with.jpg', 'height': '367'}}, 'is_blessed': True, 'uploaded_date': '2018-02-23T05:17:09-0500', 'caption': 